In [1]:
import os
import pandas as pd
import pickle
import json


from src.dataset.dataset_info import datasets
from src.dataset.clean_dataset import clean_dataset
from src.dataset.create_class_num_col import one_dataset_class_num_col
from src.dataset.features_analysis import feature_analysis_pipeline
from src.dataset.undersample_classes import undersample_classes
from src.graph.graph_measures import calculate_graph_measures
from local_variables import datasets_main_path


In [2]:
with_sort_timestamp = False
with_undersample_classes = False

name = "cic_ids_2017_5_percent"
original_path = "./testing_dfs/cic_ids_2017_5_percent.parquet"
file_type = "parquet"

# name = "cic_ton_iot_5_percent"
# original_path = "./testing_dfs/cic_ton_iot_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot"
# original_path = datasets_main_path + "cic_ton_iot.parquet"
# file_type = "parquet"

# name = "cic_ids_2017"
# original_path = datasets_main_path + "cic_ids_2017.parquet"
# file_type = "parquet"

# name = "nf_bot_iot"

# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"

folder_path = os.path.join("datasets", name)
output_path = os.path.join(folder_path, name + ".parquet")

if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

print(folder_path)
print(output_path)

datasets\cic_ids_2017_5_percent
datasets\cic_ids_2017_5_percent\cic_ids_2017_5_percent.parquet


# Preparing Datasets

### Reading and Cleaning

In [3]:
if file_type == "parquet":
    df = pd.read_parquet(original_path)
if file_type == "csv":
    df = pd.read_csv(original_path)
if file_type == "pkl":
    df = pd.read_pickle(original_path)

In [4]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack
66292,192.168.10.1-192.168.10.3-53-60671-17,192.168.10.3,60671.0,192.168.10.1,53.0,17.0,03/07/2017 10:23:37,30919.0,1.0,1.0,...,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN
244828,172.16.0.1-192.168.10.50-39546-80-6,172.16.0.1,39546.0,192.168.10.50,80.0,6.0,5/7/2017 10:55,0.0,2.0,0.0,...,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,1,DoS Hulk
420414,192.168.10.8-52.84.64.212-51938-443-6,52.84.64.212,443.0,192.168.10.8,51938.0,6.0,03/07/2017 11:20:18,3.0,1.0,1.0,...,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN
393869,172.217.10.2-192.168.10.8-80-55154-6,192.168.10.8,55154.0,172.217.10.2,80.0,6.0,5/7/2017 1:53,115525809.0,16.0,14.0,...,78974.81818,140027.3781,501173.0,36578.0,9976579.636,58066.24315,9994305.0,9801504.0,0,BENIGN
399570,None,None,NaN,None,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,None


In [5]:
timestamp_format = "mixed"
# timestamp_format="%d/%m/%Y %I:%M:%S %p"

In [6]:
df.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
       'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std',
       'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt',
       'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count',
       'ECE Flag Cnt', 'Dow

In [7]:
src_ip_col="Src IP"
src_port_col="Src Port"
dst_ip_col="Dst IP"
dst_port_col="Dst Port"
flow_id_col="Flow ID"
timestamp_col="Timestamp"
label_col="Label"
class_col="Attack"
class_num_col="Class"

In [8]:
drop_columns=[flow_id_col, src_ip_col, dst_ip_col, timestamp_col, src_port_col, dst_port_col, class_col]

In [9]:
df = clean_dataset(df, timestamp_col, flow_id_col)

In [10]:
def to_categorical_codes(column):
        return pd.Categorical(column).codes
    
if name=='x_iiot':
    columns_to_convert = [
          'Protocol', 'Service', 'is_syn_only', 'Is_SYN_ACK', 
           'is_pure_ack', 'is_with_payload', 'FIN or RST', 
        'Bad_checksum', 'is_SYN_with_RST', 'anomaly_alert']


    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
            
    df['class3'] = (df['class3'] == 'Attack').astype(int)
    df = df.applymap(lambda x: 0 if x == '-' else x)
    df = df.applymap(lambda x: 0 if x == '?' else x)
    df = df.applymap(lambda x: 0 if x == '#DIV/0!' else x)    
    df = df.applymap(lambda x: 0 if x == 'excel' else x)
    df = df.applymap(lambda x: 0 if x == 'aza' else x)
    df = df.applymap(lambda x: 0 if x == ' ' else x)
    
if name=='edge_iiot':
    columns_to_convert = ['mqtt.protoname','mqtt.topic','mqtt.conack.flags','mqtt.msg','http.request.method','dns.qry.name.len','arp.src.proto_ipv4',
                          'http.request.full_uri','http.file_data','http.request.version','arp.dst.proto_ipv4','http.request.uri.query','tcp.srcport','http.referer']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    
if name=='ccd_inid_modified':    
    columns_to_convert = ['splt_direction','splt_ps','splt_piat_ms','application_name','application_category_name','requested_server_name','client_fingerprint']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    df['traffic_type'] = (df['traffic_type'] == 'attack').astype(int)
    

In [11]:
df[src_ip_col] = df[src_ip_col].apply(str)
df[src_port_col] = df[src_port_col].apply(str)
df[dst_ip_col] = df[dst_ip_col].apply(str)
df[dst_port_col] = df[dst_port_col].apply(str)

In [12]:
_,var_dropped, corr_dropped = feature_analysis_pipeline(df=df, drop_columns=drop_columns,label_col=label_col)
var_dropped, corr_dropped

(['Bwd PSH Flags',
  'Bwd URG Flags',
  'Fwd Byts/b Avg',
  'Fwd Pkts/b Avg',
  'Fwd Blk Rate Avg',
  'Bwd Byts/b Avg',
  'Bwd Pkts/b Avg',
  'Bwd Blk Rate Avg'],
 {'Active Mean',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Duration',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow Pkts/s',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd IAT Tot',
  'Fwd PSH Flags',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Fwd URG Flags',
  'Idle Max',
  'Idle Mean',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Std',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'RST Flag Cnt',
  'Subflow Bwd Byts',
  'Subflow Bwd Pkts',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts'})

### Attacks Types

In [13]:
df[class_col].unique()

array(['BENIGN', 'PortScan', 'DoS Hulk', 'DoS slowloris', 'DDoS',
       'DoS Slowhttptest', 'FTP-Patator', 'SSH-Patator', 'DoS GoldenEye',
       'Web Attack � Brute Force', 'Infiltration', 'Bot',
       'Web Attack � XSS', 'Web Attack � Sql Injection'], dtype=object)

In [14]:
# df[class_col] = df[class_col].replace({"BENIGN": "Benign",
#                                        "DDoS": "ddos",
#                                        "Web Attack � Brute Force": "bruteforce",
#                                        "Web Attack � XSS": "xss"})

In [15]:
classes = df[class_col].unique()

### Sorting (optional)

In [16]:
if with_sort_timestamp:
    df[timestamp_col] = pd.to_datetime(df[timestamp_col].str.strip(), format=timestamp_format)
    df.sort_values(timestamp_col, inplace= True)

### Encoding Attacks into integers

In [17]:
df, labels_names = one_dataset_class_num_col(df, class_num_col, class_col)

==>> labels_names: {np.int64(0): np.str_('BENIGN'), np.int64(1): np.str_('Bot'), np.int64(2): np.str_('DDoS'), np.int64(3): np.str_('DoS GoldenEye'), np.int64(4): np.str_('DoS Hulk'), np.int64(5): np.str_('DoS Slowhttptest'), np.int64(6): np.str_('DoS slowloris'), np.int64(7): np.str_('FTP-Patator'), np.int64(8): np.str_('Infiltration'), np.int64(9): np.str_('PortScan'), np.int64(10): np.str_('SSH-Patator'), np.int64(11): np.str_('Web Attack � Brute Force'), np.int64(12): np.str_('Web Attack � Sql Injection'), np.int64(13): np.str_('Web Attack � XSS')}


### Undersampling classes (optional)

In [18]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

In [19]:
if with_undersample_classes:
    # Get the classes with the highest number of records (you can choose how many to undersample)
    classes_to_undersample = class_counts_sorted.index[:2]

    # Undersample the classes with the highest number of records
    dfs = []
    for class_label in class_counts_sorted.index:
        print(f"==>> class_label: {class_label}")
        if class_label in classes_to_undersample:
            class_df = df[df[class_col] == class_label]
            undersampled_df = class_df.sample(frac=0.5)  # Specify the fraction of samples to keep
            dfs.append(undersampled_df)
        else:
            dfs.append(df[df[class_col] == class_label])

    df = []
    # Optional: shuffle the undersampled DataFrame
    df = pd.concat(dfs).sample(frac=1).reset_index(drop=True)


In [20]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

### saving labels encodings and datasets properties

In [21]:
with open(folder_path + '/labels_names.pkl', 'wb') as f:
    pickle.dump([labels_names, classes], f)

In [22]:
total_count = len(df)

properties = {
    "name": name,
    "length": total_count,
}

num_benign = len(df[df['Label'] == 0])
num_attack = len(df[df['Label'] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df["Attack"].unique()) 

with open(folder_path + '/df_properties.json', 'w') as f:
    json.dump(properties, f)
    
properties


{'name': 'cic_ids_2017_5_percent',
 'length': 141255,
 'num_benign': 113501,
 'percentage_of_benign_records': 80.35184595235567,
 'num_attack': 27754,
 'percentage_of_attack_records': 19.648154047644333,
 'attacks': ['BENIGN',
  'PortScan',
  'DoS Hulk',
  'DoS slowloris',
  'DDoS',
  'DoS Slowhttptest',
  'FTP-Patator',
  'SSH-Patator',
  'DoS GoldenEye',
  'Web Attack � Brute Force',
  'Infiltration',
  'Bot',
  'Web Attack � XSS',
  'Web Attack � Sql Injection']}

In [23]:
# graphs_properties_path = folder_path + '/graphs_properties.json'
# G = nx.from_pandas_edgelist(
#     df,
#     source=src_ip_col,
#     target=dst_ip_col,
#     create_using=nx.DiGraph()
# )
# calculate_graph_measures(G, graphs_properties_path)

In [24]:
df.to_parquet(output_path)